In [1]:
!pip install spacy==3.0.6

In [2]:
!pip install -U spacy-nightly --pre

In [3]:
!pip install -U pip setuptools wheel

In [4]:
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 459.7 MB 1.2 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 42 kB 1.7 MB/s             
     |████████████████████████████████| 42 kB 21.9 MB/s            
     |████████████████████████████████| 40 kB 51.6 MB/s            
     |████████████████████████████████| 2.6 MB 11.3 MB/s            
     |████████████████████████████████| 1.1 MB 44.9 MB/s            
     |████████████████████████████████| 596 kB 56.3 MB/s            
     |████████████████████████████████| 3.3 MB 42.7 MB/s            
     |████████████████████████████████| 895 kB 53.2 MB/s            
  Created wheel for en-core-web-trf: filename=en_core_web_trf-3.0.0-py3-none-any.whl size=459708991 sha256=fc217ec61fd9692718e41456d12d9fd01f5830e21950fede00ccc897ba628a95
  Stored in directory: /tmp/pip-ephem-wheel-cache-72n_46xl/wheels/be/d9/92/c65214265163256ecc7588e6468c43feb86424a69bfe2fddb9
Successfully built en-core-web-trf
  Atte

In [5]:
!pip install -U spacy transformers

     |████████████████████████████████| 6.0 MB 4.7 MB/s            
     |████████████████████████████████| 3.4 MB 38.4 MB/s            
     |████████████████████████████████| 181 kB 49.6 MB/s            
     |████████████████████████████████| 61 kB 110 kB/s             
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
  Attempting uninstall: transformers
    Found existing installation: transformers 4.9.2
    Uninstalling transformers-4.9.2:
      Successfully uninstalled transformers-4.9.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.6
    Uninstalling spacy-3.0.6:
      Successfully uninstalled spacy-3.0.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.0.4 requires tr

In [6]:
!pip install spacy-transformers

     |████████████████████████████████| 2.6 MB 5.4 MB/s            
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.2.1
    Uninstalling huggingface-hub-0.2.1:
      Successfully uninstalled huggingface-hub-0.2.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.15.0
    Uninstalling transformers-4.15.0:
      Successfully uninstalled transformers-4.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-trf 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.2.1 which is incompatible.


In [10]:
%cd /content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization

/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization


In [11]:
import random

from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
#from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [12]:
nlp2 = spacy.load("/content/drive/MyDrive/TA_project/progetto_TA/training/model-best")

In [31]:
text = "Con l'ordinanza indicata in epigrafe, il Tribunale di Sorveglianza di Roma dichiarava inammissibile l'istanza di concessione della detenzione domiciliare avanzata da Gianfranco Cinti e rigettava quella di affidamento in prova al servizio sociale. ".lower() 
text

"con l'ordinanza indicata in epigrafe, il tribunale di sorveglianza di roma dichiarava inammissibile l'istanza di concessione della detenzione domiciliare avanzata da gianfranco cinti e rigettava quella di affidamento in prova al servizio sociale. "

In [16]:
import spacy.cli

spacy.cli.download('it_core_news_lg')

✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [32]:
nlp_it = spacy.load('it_core_news_lg')
documento = nlp_it(text)

In [33]:
sentenza = nlp2(documento)

In [34]:
probs = 0
imputato = ''
for value, rel_dict in sentenza._.rel.items():
    print(value)
    print(rel_dict)
                    
print(f'imputato: {imputato} con una probabilità di {probs}')                    

(12, 24)
{'DIFENDE': 0.00029228794, 'GIUDICA': 0.14727212}
(24, 12)
{'DIFENDE': 0.002916507, 'GIUDICA': 0.00022111215}
imputato:  con una probabilità di 0


In [35]:
probs = 0
imputato = ''
for value, rel_dict in sentenza._.rel.items():
    for e in sentenza.ents:
        
        for b in sentenza.ents:
            
            
            if e.start == value[0] and b.start == value[1]:
                if rel_dict['DIFENDE'] > rel_dict['GIUDICA']:
                    if rel_dict['DIFENDE'] > probs:
                        
                        probs = rel_dict['DIFENDE']
                        imputato = b.text
                    
                    
                else:
                    if rel_dict['GIUDICA'] > probs:
                        probs = rel_dict['GIUDICA']
                        imputato = b.text
                    
print(f'imputato: {imputato} con una probabilità di {probs}')                    

imputato: gianfranco cinti con una probabilità di 0.14727212488651276
